In [2]:
import sys
sys.path.append("C://Users//User//PycharmProjects//Musgrave_scripts//BenRich//common_funcs")
import cohpfuncs as cfunc
import jdftxfuncs as jfunc
#data_dir = "E://perl//pscratch//beri9208//pot_scan//PtPyrOpt150//"
data_dir = "D://scratch_backup//perl//pot_scan//AuPyrOpt//"
data_label = data_dir.split("//")[-2]
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from numba import jit
savedir = "C://Users//User//Desktop//pcoop_plots//bandwise//"
data_prefix = "output."

def app_dir(surf_idx, mu_idx):
    """
    :param surf_idx: 0 --> 2n-cis, 1 --> 2n-trans
    :param mu_idx: 0 --> No_bias, -1 --> dneg/1, 1 --> 2pos/1
    :return:
    """
    out_str = surf_labels[surf_idx] + "//"
    if mu_idx == 0:
        out_str += "No_bias//"
    else:
        if mu_idx > 0:
            out_str += "dpos//"
        else:
            out_str += "dneg//"
        out_str += str(int(abs(mu_idx))) + "//"
    return data_dir + out_str

surf_labels = ["2n-cis", "2n-trans", "3n", "4n"]
bias_labels = ["-0.3V", "0.0V", "0.3V", "0.6", "0.9V", "1.2V", "1.5V", "1.8V", "-2.4V", "-2.1V", "-1.8V", "-1.5V", "-1.2V", "-0.9V", "-0.6V"]
bidrange = [-7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7]
bshift = int(np.floor(len(bidrange)/2.))
nbids = len(bidrange)
ncalcs = nbids*2

def gen_dirname(surf_idx, bias_idx):
    slabel = surf_labels[surf_idx]
    out_str = save_dir + slabel + str(bias_idx)
    out_str += '//'
    return out_str

def gen_savedirname(surf_idx, savedir):
    slabel = surf_labels[surf_idx]
    out_str = savedir + slabel
    out_str += '//'
    return out_str

def setupdir(dirname):
    try:
        os.mkdir(dirname)
    except:
        None
setupdir(savedir)


def gen_savename(surf_idx, bias_idx, j):
    out_str = str(j) + "_" + str(bias_idx) +  surf_labels[surf_idx]
    return out_str

def gen_title(surf_idx, bias_idx, j):
    out_str = str(j) + " " + bias_labels[bias_idx - bshift] + " " +  surf_labels[surf_idx]
    return out_str

@jit(nopython=True)
def lb_idx(Emin, dE, sample):
    a = sample-Emin
    b = a/dE
    c = int(np.floor(b))
    return c, abs(b-c)

def get_ebounds(surf_idcs, bias_idcs):
    minE = 0
    maxE = 0
    for sidx in surf_idcs:
        for bidx in bias_idcs:
            output = cfunc.parse_data(app_dir(sidx, bidx) + data_prefix + "bandProjections", app_dir(sidx, bidx) + data_prefix + "Gvectors", app_dir(sidx, bidx) + data_prefix + "eigenvals")
            minE = min(minE, np.min(np.real(output[-1])))
            maxE = max(maxE, np.min(np.real(output[-1])))
    return minE, maxE

def get_erange(surf_idcs, bias_idcs, dE):
    minE, maxE = get_ebounds(surf_idcs, bias_idcs)
    minE += -5*dE
    maxE += 5*dE
    return np.arange(minE, maxE, dE)

@jit(nopython=True)
def get_overlap_matrix_plottable_smear_helper(ejk, height, Erange, outij, dE, sw):
    a = ejk - min(Erange)
    b = a / dE
    idx = int(np.floor(b))
    for i in range(sw*5 + 1):
        act = idx + i - sw
        outij[act] += height*np.exp(-((Erange[act] - ejk)**2)/(sw*dE))

@jit(nopython=True)
def fiterable(orbi, orbj, nk, nj, E, proj, out, wk, Erange, dE, sw, i, j):
    for u1 in orbi:
            if u1 > 0:
                for v1 in orbj:
                    if v1 > 0:
                        for ik in range(nk):
                            for ij in range(nj):
                                ejk = E[ik][ij]
                                t1 = proj[ik][ij][u1]
                                t2 = proj[ik][ij][v1]
                                t1_conj = t1.real - t1.imag * 1j
                                height = np.real(t1_conj * t2 * wk[ik])
                                get_overlap_matrix_plottable_smear_helper(ejk, height, Erange, out[i,j], dE, sw)

@jit(nopython=True)
def get_overlap_matrix_plottable_smear(proj, E, wk, orbs_sample, nk, nj, out, Erange, dE, sw):
    # Fix me eventually
    # print('starting2')
    for ii in list(range(len(orbs_sample))):
        if ii == 37:
            for jj in range(len(orbs_sample)):
                if jj == 40:
                    orbi = orbs_sample[ii]
                    orbj = orbs_sample[jj]
                    fiterable(orbi, orbj, nk, nj, E, proj, out, wk, Erange, dE, sw, ii, jj)
        else:
            for jj in range(len(orbs_sample)):
                orbi = orbs_sample[ii]
                orbj = orbs_sample[jj]
                fiterable(orbi, orbj, nk, nj, E, proj, out, wk, Erange, dE, sw, ii, jj)
    return out

@jit(nopython=True)
def get_band_vector_nosmear_helper(ejk, height, Erange, outij, dE):
    a = ejk - min(Erange)
    b = a / dE
    idx = int(np.floor(b))
    spill = abs(b-idx)
    outij[idx] += height*(1 - spill)
    outij[idx+1] += height*spill

@jit(nopython=True)
def get_band_vector_plottable_nosmear(E, wk, nk, nj, out, Erange, dE):
    for ij in range(nj):
        for ik in range(nk):
            ejk = E[ik][ij]
            height = wk[ik]
            get_band_vector_nosmear_helper(ejk, height, Erange, out[ij], dE)
    return out


@jit(nopython=True)
def get_plot_stuff(sidx, bidx, j, ndatum):
    dlook = ndatum[sidx][bidx]
    y = np.real(dlook[j])
    return y

def system_bandwise_plottable(surf_idx, bias_idx, Erange, dE):
    proj, nStates, nBands, nProj, nOrbsPerAtom, wk, k_points, E = cfunc.parse_data(
        app_dir(surf_idx, bias_idx) + data_prefix + "bandProjections",
        app_dir(surf_idx, bias_idx) + data_prefix + "Gvectors",
        app_dir(surf_idx, bias_idx) + data_prefix + "eigenvals")
    S, R, mu = jfunc.get_vars(app_dir(surf_idx, bias_idx) + "out")
    nk, nj = np.shape(E)
    print(nj)
    out = np.zeros([nj, len(Erange)])
    band_vec = get_band_vector_plottable_nosmear(E, wk, nk, nj, out, Erange, dE)
    """
    pcoops_list: 2d list
    Erange: 1d range
    titles_and_idcs_and_mu: [title, data i/j pair, mu]
    """
    return band_vec, mu

In [ ]:
surfs = 4
dE = 0.01
Erange = get_erange([0, 1, 2, 3], bidrange, dE)
datum = []
for i in range(surfs):
    datum.append([])
    for j in range(len(bias_labels)):
        start = time.time()
        datum[-1].append(system_bandwise_plottable(i, j - bshift, Erange, dE))
        end = time.time()
        elapsed = end - start
        print(str(i*len(bias_labels) + j + 1) + "/" + str(2*len(bias_labels)) + ": " + f'{elapsed:.{4}g}' )

150


C:\Users\User\AppData\Roaming\Python\Python310\site-packages\numba\core\ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'wk' of function 'get_band_vector_plottable_nosmear'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_13224\3915694250.py", line 142:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


1/30: 1.676
150
2/30: 0.569


In [ ]:
mus = []
for bidx in range(len(bias_labels)):
    mus.append(datum[0][bidx][1])
nj = len(datum[0][0][0])
ndatum = np.zeros([np.shape(datum)[0], len(bidrange), nj, len(Erange)])
for sidx in range(surfs):
    for bidx in range(len(bidrange)):
        for j in range(nj):
            ndatum[sidx, bidx, j] += np.real(datum[sidx][bidx][0][j])

In [ ]:
y_bounds = np.zeros([len(surf_labels), nj, 2])
for sidx in range(surfs):
    for j in range(nj):
        miny = 0
        maxy = 0
        for bidx in range(len(bidrange)):
            miny = min(miny, np.nanmin(ndatum[sidx][bidx][j]))
            maxy = max(maxy, np.nanmax(ndatum[sidx][bidx][j]))
        y_bounds[sidx, j, 0] += miny
        y_bounds[sidx, j, 1] += maxy

def saveplot(sidx, bidx, j, savedir):
    y = get_plot_stuff(sidx, bidx, j, ndatum)
    plt.figure(figsize=(10,5))
    plt.plot(Erange, y)
    plt.axvline(x=mus[bidx], color='red')
    plt.title(gen_title(sidx, bidx, j))
    yb = y_bounds[sidx, j]
    plt.ylim(yb[0], yb[1])
    plt.savefig(gen_savedirname(sidx, savedir) + gen_savename(sidx, bidx, j) + '.png')
    plt.clf()

In [ ]:
sidx = 0
setupdir(gen_savedirname(sidx, savedir))
for bidx in range(len(bidrange)):
    start = time.time()
    for j in range(nj):
        saveplot(sidx, bidx, j, savedir)
    end = time.time()
    elapsed = end - start
    print(str(sidx*len(bidrange) + bidx + 1) + "/" + str(len(bidrange)*2) + ": " + f'{elapsed:.{4}g}' + 'seconds')

In [ ]:
sidx = 1
setupdir(gen_savedirname(sidx, savedir))
for bidx in range(len(bidrange)):
    start = time.time()
    for j in range(nj):
        saveplot(sidx, bidx, j, savedir)
    end = time.time()
    elapsed = end - start
    print(str(sidx*len(bidrange) + bidx + 1) + "/" + str(len(bidrange)*2) + ": " + f'{elapsed:.{4}g}' + 'seconds')

In [ ]:
sidx = 2
setupdir(gen_savedirname(sidx, savedir))
for bidx in range(len(bidrange)):
    start = time.time()
    for j in range(nj):
        saveplot(sidx, bidx, j, savedir)
    end = time.time()
    elapsed = end - start
    print(str(sidx*len(bidrange) + bidx + 1) + "/" + str(len(bidrange)*2) + ": " + f'{elapsed:.{4}g}' + 'seconds')

In [ ]:
sidx = 1
setupdir(gen_savedirname(sidx, savedir))
for bidx in range(len(bidrange))[:int(len(bidrange)/2)]:
    start = time.time()
    for ai in range(natoms):
        for aj in list(range(natoms))[ai:]:
            ai_use = natoms - ai - 1
            aj_use = natoms - aj - 1
            saveplot(sidx, bidx, ai_use, aj_use, savedir)
    end = time.time()
    elapsed = end - start
    print(str(sidx*len(bidrange) + bidx + 1) + "/" + str(len(bidrange)*2) + ": " + f'{elapsed:.{4}g}' + 'seconds')

In [ ]:
sidx = 1
setupdir(gen_savedirname(sidx, savedir))
for bidx in range(len(bidrange))[int(len(bidrange)/2):]:
    start = time.time()
    for ai in range(natoms):
        for aj in list(range(natoms))[ai:]:
            ai_use = natoms - ai - 1
            aj_use = natoms - aj - 1
            saveplot(sidx, bidx, ai_use, aj_use, savedir)
    end = time.time()
    elapsed = end - start
    print(str(sidx*len(bidrange) + bidx + 1) + "/" + str(len(bidrange)*2) + ": " + f'{elapsed:.{4}g}' + 'seconds')